In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime, timedelta
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
import random
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
from tensorflow.math import erf
from scipy.stats import norm
from sklearn.preprocessing import Normalizer,StandardScaler, LabelEncoder
from tensorflow_addons.losses import pinball_loss
from tensorflow_lattice.layers import Linear

# Read and preprocess data

In [2]:
data = pd.read_feather("data/berlin_data/historic_data/icon_eps_wind_10m.feather")
data.head()

,init_tm,met_var,location,fcst_hour,obs_tm,obs,ens_1,ens_2,ens_3,ens_4,...,ens_33,ens_34,ens_35,ens_36,ens_37,ens_38,ens_39,ens_40,ens_mean,ens_var
0,2018-12-18 00:00:00+00:00,wind_10m,Berlin,0.0,2018-12-18 00:00:00+00:00,6.48,3.80,6.56,4.54,5.05,...,5.59,5.45,5.30,4.47,5.99,3.48,4.92,5.09,4.58675,0.565448
1,2018-12-18 00:00:00+00:00,wind_10m,Berlin,1.0,2018-12-18 01:00:00+00:00,6.12,3.68,7.03,5.06,5.33,...,4.92,5.18,4.98,4.88,6.39,3.74,5.18,4.85,4.69750,0.663747
2,2018-12-18 00:00:00+00:00,wind_10m,Berlin,2.0,2018-12-18 02:00:00+00:00,4.32,3.28,7.10,5.39,5.44,...,4.91,4.88,5.20,4.80,6.66,4.14,5.05,4.80,4.81650,0.830100
3,2018-12-18 00:00:00+00:00,wind_10m,Berlin,3.0,2018-12-18 03:00:00+00:00,5.04,3.47,7.45,6.11,5.66,...,5.37,4.58,5.30,4.86,6.96,4.41,5.26,4.74,5.01625,1.222111
4,2018-12-18 00:00:00+00:00,wind_10m,Berlin,4.0,2018-12-18 04:00:00+00:00,6.48,3.56,8.02,6.29,5.81,...,5.30,4.05,5.04,5.26,6.67,4.86,4.95,4.56,5.01600,1.355050


## Dropna

In [3]:
data.dropna(inplace = True)

In [4]:
# Normalize
hour = data["fcst_hour"]
data = data.iloc[:,3:-2].drop("obs_tm", axis = 1)
data["fcst_hour"] = hour
data_np = data.to_numpy()

## Train, val, test split

In [5]:
train_val, test = train_test_split(data_np, test_size = 0.1)
train, val = train_test_split(train_val, test_size = 0.2)

### Normalize

In [6]:
def normalize(data, feature_scaler = None, target_scaler = None, learn = False):
    if learn == True:
        feature_scaler = StandardScaler()
        target_scaler = StandardScaler()
        #Learn target scaling
        target_scaled = target_scaler.fit_transform(data[:,1].reshape(-1,1))
        #Learn feature scaling
        feature_scaled = feature_scaler.fit_transform(data[:,2:])
        #Append
        data[:,1] = target_scaled.reshape(-1)
        data[:,2:] = feature_scaled
        
        return data, feature_scaler, target_scaler
    
    else:
        #Scale target
        target_scaled = target_scaler.transform(data[:,1].reshape(-1,1))
        #Scale features
        feature_scaled = feature_scaler.transform(data[:,2:])
        #Append
        data[:,1] = target_scaled.reshape(-1)
        data[:,2:] = feature_scaled
        
        return data

In [7]:
train, feature_scaler, target_scaler = normalize(train, learn = True)
train_val = normalize(train_val, feature_scaler, target_scaler)
test = normalize(test, feature_scaler, target_scaler)
val = normalize(val, feature_scaler, target_scaler)

In [8]:
def convert_format(input_data, predict = False):
    #Extract forecast embedding
    horizon_emb = input_data[:,0]
    
    if predict == False:        
        #Extract features
        features = input_data[:,2:]
        # Extract target
        target = np.expand_dims(input_data[:,1],1)
        return [features, horizon_emb], target
    else:
        #Extract features
        features = input_data[:,1:]
        return [features, horizon_emb]    

In [9]:
train_data, train_target = convert_format(train)
val_data, val_target = convert_format(val)
test_data, test_target = convert_format(test)

# Create Model

In [10]:
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
BATCH_SIZE = 512
EPOCHS = 50
learning_rate = 0.01

In [11]:
class base_model(tf.keras.Model):    
    def __init__(self, no_features = 40, n_embeddings = 121):
        super(base_model, self).__init__()
        #Embedding layers
        self.embedding = Embedding(input_dim = n_embeddings, output_dim = 4)
        #Create Dense layers
        self.hidden = Dense(25, activation = "relu")
        self.out = Dense(1, activation = "linear")

    def call(self, input_data):
        #Extract data
        features, horizon_emb = input_data
        #Calculate embedding
        emb = self.embedding(horizon_emb)
        emb = tf.squeeze(emb, axis = 1)
        conc = Concatenate(axis = 1)([features, emb])
        #Calculate output
        output = self.hidden(conc)
        output = self.out(output)

        return output

In [12]:
def train_model(model, quantile, train_data, train_target, validation_data, batch_size, epochs, learning_rate):
    #Define optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    #Early stopping
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5, min_delta = 1e-5)
    #Compile model
    model.compile(optimizer = optimizer, loss = lambda true,pred: pinball_loss(true, pred, tau = quantile))
    model.fit(x = train_data, y = train_target, validation_data = validation_data, epochs = epochs, batch_size = batch_size, callbacks = [callback], shuffle = True, verbose = False)

In [13]:
def create_all_models(train_data, train_target, validation_data, batch_size = BATCH_SIZE, epochs = EPOCHS, learning_rate = learning_rate, quantiles = quantiles):
    models = []
    for quantile in quantiles:
        model = base_model()
        train_model(model, quantile, train_data, train_target, validation_data, batch_size, epochs, learning_rate)
        print("Training finished for quantile: {}".format(quantile))
        models.append(model)
    return models

In [14]:
trained_models = create_all_models(train_data, train_target, (val_data, val_target))

Training finished for quantile: 0.025
Training finished for quantile: 0.25
Training finished for quantile: 0.5
Training finished for quantile: 0.75
Training finished for quantile: 0.975


# Predict test data

## Evaluate pinball loss

In [15]:
predictions = []
for cnt,_ in enumerate(quantiles):
    pred = trained_models[cnt].predict(test_data)
    predictions.append(pred)

In [16]:
for cnt,quantile in enumerate(quantiles):
    loss = pinball_loss(test_target.reshape(-1), predictions[cnt].reshape(-1), tau = quantile).numpy()
    print("Pinball loss for quantile {} : \t {}".format(quantile,loss))

Pinball loss for quantile 0.025 : 	 0.030550554394721985
Pinball loss for quantile 0.25 : 	 0.17490561306476593
Pinball loss for quantile 0.5 : 	 0.22989022731781006
Pinball loss for quantile 0.75 : 	 0.19356249272823334
Pinball loss for quantile 0.975 : 	 0.03985196352005005


## Evaluate pinball loss on naive prediction

In [17]:
naive_pred = np.quantile(test_data[0], quantiles, axis = 1)
for cnt,quantile in enumerate(quantiles):
    loss = pinball_loss(np.squeeze(test_target), naive_pred[cnt], tau = quantile).numpy()
    print("Pinball loss for quantile {} : \t {}".format(quantile,loss))

Pinball loss for quantile 0.025 : 	 0.06182668176195165
Pinball loss for quantile 0.25 : 	 0.19513981675276262
Pinball loss for quantile 0.5 : 	 0.23807509441740377
Pinball loss for quantile 0.75 : 	 0.20293556067357782
Pinball loss for quantile 0.975 : 	 0.07250960824050695


# Predict new data

## Train on complete data without test set

In [18]:
train_data, train_target = convert_format(train_val)
val_data, val_target = convert_format(test)

In [19]:
trained_models = create_all_models(train_data, train_target, (val_data, val_target))

Training finished for quantile: 0.025
Training finished for quantile: 0.25
Training finished for quantile: 0.5
Training finished for quantile: 0.75
Training finished for quantile: 0.975


## Predict new data

In [20]:
horizons = [36, 48 ,60, 72, 84]

In [23]:
#Set current date
current_date = date.today().strftime("%Y%m%d")
path = "data/berlin_data/icon_data/icon-eu-eps_{}00_wind_mean_10m_Berlin.txt".format(current_date)
new_data = pd.read_csv(path.format(current_date.replace("-","")), skiprows = 3, sep = "|").dropna(axis = 1)
new_data.columns = new_data.columns.str.replace(" ", "")
# Normalize and get horizons
new_data = new_data[new_data["fcst_hour"].isin(horizons)].to_numpy()
new_data[:,1:] = feature_scaler.transform(new_data[:,1:])

In [24]:
pred_data = convert_format(new_data, predict = True)

In [25]:
#Prepare dataframe
final_prediction = pd.DataFrame(columns = ["forecast_date","target","horizon","q0.025","q0.25","q0.5","q0.75","q0.975"], index = np.arange(0,5))
final_prediction["forecast_date"] = datetime.today().strftime("%Y-%m-%d")
final_prediction["horizon"] = ["{} hour".format(x) for x in horizons]
final_prediction["target"] = "wind"

In [27]:
for cnt, quantile in enumerate(quantiles):
    #Get prediction
    prediction = trained_models[cnt].predict(pred_data)
    #Retransform
    final_pred = target_scaler.inverse_transform(prediction)
    final_prediction.loc[:,"q{}".format(quantile)] = final_pred

In [28]:
final_prediction

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2021-11-03,wind,36 hour,9.008980,14.112084,17.420645,20.307768,25.082108
1,2021-11-03,wind,48 hour,8.901674,14.817027,17.945885,21.549999,28.956566
2,2021-11-03,wind,60 hour,9.045424,13.840762,16.221087,18.463316,23.152771
3,2021-11-03,wind,72 hour,7.066521,12.672591,14.257113,17.415918,25.860399
4,2021-11-03,wind,84 hour,10.816050,15.574940,19.018330,21.133451,26.780411


In [41]:
final_prediction.to_pickle("../evaluation/predictions/single/{}_{}".format("wind", date.today().strftime("%Y-%m-%d")))